# Pre-requisites (optional but recommended)
Only do the first step if you have never created a virtual environment for this repository. Otherwise, make sure that the Python Kernel that you selected is from your venv/ folder.

In [2]:
! source ../venv/bin/activate

In [35]:
! python3 -V
! which python3

Python 3.13.11
/Users/jcheng/Documents/ljcheng/ml/learning/repos/rag-langchain-agent/venv/bin/python3


# Topics Covered:
- LangChain LLM Basics
- LLM Invocation
- LLM with Tools
- Structured Output from LLM
- Basic LangGraph Chatbot
- Adding Memory to the Chatbot
- LangGraph Agent with Tools
- LangGraph RAG Agent

### Part 1: Environment and Packages

In [100]:
! pip3 install -q --upgrade pip
! pip3 install -qU langchain langchain-openai langchain-community langchain-core python-dotenv python-multipart

In [127]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

# Access the environment variables
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

if(not os.getenv('LANGCHAIN_API_KEY')):
  raise KeyError("set LANGCHAIN_API_KEY")
if(not os.getenv('OPENAI_API_KEY')):
  raise KeyError("set OPENAI_API_KEY")
   

In [102]:
from langchain_openai import ChatOpenAI
# Initialize the LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)

In [122]:
from langchain_core.messages import HumanMessage, SystemMessage
# Using messages for more control
messages = [
  SystemMessage(content="You are a helpful AI assistant that explains complex topics simply."),
  HumanMessage(content="Explain machine learning in 2 sentences.")
]

response = llm.invoke(messages)
response

AIMessage(content='Machine learning is a type of artificial intelligence that enables computers to learn from data and improve their performance on tasks without being explicitly programmed. It involves training algorithms to recognize patterns and make predictions or decisions based on new input.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D68QcoTkKjeu6d1OvUOQ4zZ61PFwG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c315d-819d-7e70-8cb6-2d0b3fc48688-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 31, '

# LLM with Tools

In [123]:

# Free to use, refer other infor here: https://docs.langchain.com/oss/python/integrations/tools/google_serper

In [105]:
! pip3 freeze > ../requirements.txt

In [160]:
from langchain_core.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper
## https://docs.langchain.com/oss/python/integrations/tools/google_serper

search = GoogleSerperAPIWrapper() # Initialize an instance before the decorator tool under internet_serper_search()
# search = GoogleSerperAPIWrapper(type="news")

In [161]:
@tool
def calculator(expression: str) -> str:
  """Calculate mathematical expressions. Use this for any math calculations."""
  try:
    result = eval(expression)
    return f"The result of {expression} is {result}"
  except Exception as e:
    return f"Error calculating {expression}: {str(e)}"

@tool
def internet_serper_search(query: str) -> str:
  """Useful for when you need to ask with search."""
  return search.run(query)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools([calculator, internet_serper_search])

# Test the calculator tool
print("Testing Calculator Tool:")
response = llm_with_tools.invoke("What's 25 * 4 + 17?")
print(f"Response: {response.content}")

Testing Calculator Tool:
Response: 


LLM decided to use the calculator tool instead of the search_tool

In [157]:
response
response.tool_calls

[{'name': 'calculator',
  'args': {'expression': '25 * 4 + 17'},
  'id': 'call_lLvH1Cy9f211eZkMjig9DusJ',
  'type': 'tool_call'}]

In [163]:
# Map tool names to tool objects for dynamic execution
tool_map = {
  'calculator': calculator,
  'internet_serper_search': internet_serper_search,
}

def handle_tool_calls(response, tool_map):
  """Executes all tool calls in the LLM response using the tool_map."""
  if not getattr(response, 'tool_calls', None):
    return

  print(f"Tool calls requested: {len(response.tool_calls)}")
  for tool_call in response.tool_calls:
    tool_name = tool_call['name']
    args = tool_call['args']
    print(f"Tool: {tool_name}")
    print(f"Args: {args}")

    tool = tool_map.get(tool_name)
    if tool:
      result = tool.invoke(args)
      # Print first 200 chars for long responses (e.g., search)
      preview = result[:200] + "..." if isinstance(result, str) and len(result) > 200 else result
      print(f"Tool result: {preview}")

In [ ]:

def test_llm_tool(query):
  print(f"Query: {query}")
  response = llm_with_tools.invoke(query)
  print(f"Response: {getattr(response, 'content', response)}")
  handle_tool_calls(response, tool_map)
  print("\n")

test_llm_tool("What's 25 * 4 + 17?")
test_llm_tool("Search for recent news about artificial intelligence")

Query: What's 25 * 4 + 17?
Response: 
Tool calls requested: 1
Tool: calculator
Args: {'expression': '25 * 4 + 17'}
tool:  name='calculator' description='Calculate mathematical expressions. Use this for any math calculations.' args_schema=<class 'langchain_core.utils.pydantic.calculator'> func=<function calculator at 0x1111f3240>
Tool result: The result of 25 * 4 + 17 is 117


Query: Search for recent news about artificial intelligence
Response: 
Tool calls requested: 1
Tool: internet_serper_search
Args: {'query': 'recent news about artificial intelligence'}
tool:  name='internet_serper_search' description='Useful for when you need to ask with search.' args_schema=<class 'langchain_core.utils.pydantic.internet_serper_search'> func=<function internet_serper_search at 0x1111f3b00>
Tool result: AI News delivers the latest updates in artificial intelligence, machine learning, deep learning, enterprise AI, and emerging tech worldwide. The World's First Viral AI Assistant Has Arrived, and Thi

# STRUCTURED OUTPUT FROM LLM

In [165]:
from pydantic import BaseModel, Field
from typing import List, Optional

class PersonInfo(BaseModel):
  """Information about a person"""
  name: str = Field(description="Full name of the person")
  age: Optional[int] = Field(description="Age of the person")
  occupation: str = Field(description="Person's job or profession")
  skills: List[str] = Field(description="List of skills or expertise")

structured_llm = llm.with_structured_output(PersonInfo)

# Test with person information
print("Testing Structured Output - Person Info:")
person_prompt = """
Extract information about this person:
"John Smith is a 35-year-old software engineer who works at Google.
He specializes in machine learning, Python programming, and cloud architecture.
John has been working in tech for over 10 years and is passionate about AI research."
"""

person_result = structured_llm.invoke(person_prompt)
print(f"Name: {person_result.name}")
print(f"Age: {person_result.age}")
print(f"Occupation: {person_result.occupation}")
print(f"Skills: {', '.join(person_result.skills)}")

Testing Structured Output - Person Info:
Name: John Smith
Age: 35
Occupation: Software engineer
Skills: machine learning, Python programming, cloud architecture


# Part 2: Basic LangGraph Chatbot

In [167]:
!pip3 install -qU langgraph

# LangGragh State

In [169]:
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages

In [170]:
class State(TypedDict):
  """State for our chatbot - this holds the conversation history"""
  # The add_messages function handles appending new messages to the conversation
  messages: Annotated[list[BaseMessage], add_messages]

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)

# CREATING THE CHATBOT NODE

In [ ]:
def chatbot_node(state: State) -> State:
  """
  The main chatbot node that processes messages and generates responses
  """
  print(f"Processing {len(state['messages'])} messages")

  # Get the response from the LLM
  response = llm.invoke(state["messages"])

  # Return the updated state with the new response
  return {"messages": [response]}

print("Chatbot node function created")